In [33]:
from google.colab import drive
import shutil
import os
def copy_from_drive(src_path, dst_path):

    if os.path.exists(dst_path):
        print(f"skip:{dst_path} exists")
        return

    if os.path.isdir(src_path):
        shutil.copytree(src_path, dst_path)
    elif os.path.isfile(src_path):
        shutil.copy(src_path, dst_path)

drive.mount('/content/drive')
copy_from_drive('/content/drive/MyDrive/tool', '/content/tool')
copy_from_drive('/content/drive/MyDrive/MicroLens-50k_pairs.csv','/content/MicroLens-50k_pairs.csv')
copy_from_drive('/content/drive/MyDrive/cover_emb128.lmdb','/content/cover_emb128.lmdb')
copy_from_drive('/content/drive/MyDrive/title_emb1024.lmdb','/content/title_emb1024.lmdb')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
skip:/content/tool exists
skip:/content/MicroLens-50k_pairs.csv exists
skip:/content/cover_emb128.lmdb exists


In [34]:
!pip install faiss-cpu
!pip install lmdb
from tool import preprocess
from tool import customdataset
from tool import evaluate
import faiss
from datetime import datetime
import scipy.sparse as sp
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd



In [35]:
preprocess.set_seed(42)

<function tool.preprocess.set_seed.<locals>.seed_worker(worker_id)>

In [36]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [37]:
# dataset_pd = pd.read_csv('D:\\VideoRecSystem\\MicroLens\\DataSet\\MicroLens-50k_pairs.csv')
path = 'MicroLens-50k_pairs.csv'
cover_lmdb_path = 'cover_emb128.lmdb'
title_lmdb_path = 'title_emb1024.lmdb'
user = 'user'
item = 'item'
user_id = 'user_id'
item_id = 'item_id'
timestamp = 'timestamp'
save_dir = './embeddings'
N_LAYERS = 2
EMBEDDING_DIM = 64
EPOCHS = 8
BATCH_SIZE = 1024
LR = 1E-3
MODAL = {'COVER':{"LMDB_DIM":128, "HIDDEN_SIZE":[EMBEDDING_DIM],"DROPOUT":0.2} , 'TITLE':{"LMDB_DIM":1024,"HIDDEN_SIZE":[EMBEDDING_DIM],"DROPOUT":0.2}
         ,'COVER-TITLE': {"LMDB_DIM":128+1024, "HIDDEN_SIZE":[EMBEDDING_DIM],"DROPOUT":0.2}}
CURRENT_MODAL = "COVER"
MODAL_CONFIG = MODAL[CURRENT_MODAL]
MODAL_HIDDEN_SIZE = MODAL_CONFIG.get('HIDDEN_SIZE')
LMDB_DIM = MODAL_CONFIG.get('LMDB_DIM')
MODAL_DROPOUT = MODAL_CONFIG.get('DROPOUT')
FUSION_MODE = "late"
DROPOUT = 0.2
L2_NORM = False
top_k = 10
num_workers = 10
k_neg = 10
# path = pd.read_csv('MicroLens-50k_pairs.csv')
# dataset_pd = pd.read_csv('MicroLens-50k_pairs.csv')

In [38]:
dataset_pd,num_users,num_items = preprocess.openAndSort(path,user_id=user,item_id=item,timestamp='timestamp')

dataset base information：
- number of users：50000
- number of items：19220
- number of rows：359708


In [39]:

train_df, test_df = preprocess.split(dataset_pd,user, item, timestamp)
print(f"Train size: {len(train_df)}")
print(f"Test size: {len(test_df)}")

Train size: 309708
Test size: 49424


In [40]:
# maintain a map from new id to old id, new id for constructing matrix
user2id = {u: i for i, u in enumerate(dataset_pd[user].unique())}
item2id = {i: j for j, i in enumerate(dataset_pd[item].unique())}

# apply to train_df and test_df
train_df[user_id] = train_df[user].map(user2id)
train_df[item_id] = train_df[item].map(item2id)
test_df[user_id] = test_df[user].map(user2id)
test_df[item_id] = test_df[item].map(item2id)

# 1. 构建 item_id 到 item 的映射（来自 train_df）
item_id_to_item = {v: k for k, v in item2id.items()}

In [41]:

def build_adj_matrix(df, num_users, num_items ,user_id, item_id):
    rows = df[user_id].values
    cols = df[item_id].values
    data = np.ones(len(df))
    # set interaction of user-item as 1, other as 0
    R = sp.coo_matrix((data, (rows, cols)), shape=(num_users, num_items))

    # construct symetric matrix A
    upper = sp.hstack([sp.csr_matrix((num_users, num_users)), R])
    lower = sp.hstack([R.T, sp.csr_matrix((num_items, num_items))])
    A = sp.vstack([upper, lower])

    # normalization A → Ĥ = D^{-1/2} A D^{-1/2}
    rowsum = np.array(A.sum(1)).flatten()
    d_inv_sqrt = np.power(rowsum, -0.5)
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    D_inv_sqrt = sp.diags(d_inv_sqrt)
    A_norm = D_inv_sqrt @ A @ D_inv_sqrt

    # transform to torch.sparse
    A_norm = A_norm.tocoo()
    indices = torch.LongTensor([A_norm.row, A_norm.col])
    values = torch.FloatTensor(A_norm.data)
    return torch.sparse_coo_tensor(indices, values, A_norm.shape)


In [42]:
adj_torch = build_adj_matrix(train_df, num_users, num_items ,user_id, item_id)

/tmp/ipython-input-2216191345.py:15: RuntimeWarning: divide by zero encountered in power
  d_inv_sqrt = np.power(rowsum, -0.5)


In [43]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class LightGCN(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim, n_layers, adjacency,
                 lmdb_dim=LMDB_DIM, modal_hidden_size=MODAL_HIDDEN_SIZE, modal_dropout=MODAL_DROPOUT,
                 fusion_mode=FUSION_MODE):  # 'none' | 'early' | 'late'
        super(LightGCN, self).__init__()
        assert fusion_mode in {'none', 'early', 'late'}
        self.fusion_mode   = fusion_mode
        self.user_emb      = None
        self.item_emb      = None
        self.num_users     = num_users
        self.num_items     = num_items
        self.embedding_dim = embedding_dim
        self.n_layers      = n_layers
        self.adjacency     = adjacency  # torch.sparse_coo_tensor

        # ----- ID embeddings -----
        self.embedding_user = nn.Embedding(num_users, embedding_dim)
        self.embedding_item = nn.Embedding(num_items, embedding_dim)
        nn.init.xavier_uniform_(self.embedding_user.weight)
        nn.init.xavier_uniform_(self.embedding_item.weight)

        # modal 向量（冻结）
        modal_emb_tensor = None
        if CURRENT_MODAL=='COVER':
            modal_emb_tensor = preprocess.load_tensor_from_lmdb(
                cover_lmdb_path, num_items, item_id_to_item, lmdb_dim
            )
        if CURRENT_MODAL=='TITLE':
            modal_emb_tensor = preprocess.load_tensor_from_lmdb(
                title_lmdb_path, num_items, item_id_to_item, lmdb_dim
            )
        if CURRENT_MODAL=='COVER-TITLE':
            cover_emb_tensor = preprocess.load_tensor_from_lmdb(
                cover_lmdb_path, num_items, item_id_to_item, 128
            )
            title_emb_tensor = preprocess.load_tensor_from_lmdb(
                title_lmdb_path, num_items, item_id_to_item, 1024
            )
            modal_emb_tensor = torch.cat([cover_emb_tensor, title_emb_tensor], dim=-1)

        self.register_buffer('frozen_extra_emb', modal_emb_tensor)


        # ----- 前融合投影：[item_id_emb; modal] -> emb_dim -----
        self.mlp_item_modal = self.build_mlp(embedding_dim + lmdb_dim, modal_hidden_size, modal_dropout)

        # ----- 后融合用 α（全局标量）-----
        # sigmoid(0)=0.5；如需更稳可改为 1.0 使初期更偏向 ID
        self.alpha_param = nn.Parameter(torch.tensor(0.0))

    def build_mlp(self, input_dim, hidden_sizes, dropout):
        layers = []
        for h in hidden_sizes:
            layers += [nn.Linear(input_dim, h), nn.BatchNorm1d(h), nn.Tanh(), nn.Dropout(dropout)]
            input_dim = h
        return nn.Sequential(*layers)

    # ============ LightGCN 传播（给定初始 user/item 表示） ============
    @torch.no_grad()
    def _propagate(self, user_init, item_init):
        """
        user_init: (U, D), item_init: (I, D)
        返回经 n_layers LightGCN 平均聚合后的 (user_emb, item_emb)
        """
        all_embeddings = torch.cat([user_init, item_init], dim=0)   # (U+I, D)
        embs = [all_embeddings]
        for _ in range(self.n_layers):
            all_embeddings = torch.sparse.mm(self.adjacency, all_embeddings)
            embs.append(all_embeddings)
        final = torch.stack(embs, dim=1).mean(dim=1)                # (U+I, D)
        user_embedding, item_embedding = torch.split(final, [self.num_users, self.num_items])
        return user_embedding, item_embedding

    def _item_init_id_only(self):
        return self.embedding_item.weight                             # (I, D)

    def _item_init_early(self):
        # [id_emb; modal] -> emb_dim
        modal = self.frozen_extra_emb.to(self.embedding_item.weight.device)  # (I, C)
        i_cat = torch.cat([self.embedding_item.weight, modal], dim=-1)       # (I, D+C)
        i_emb = self.mlp_item_modal(i_cat)                                   # (I, D)
        return i_emb

    # ============ 前向：生成/缓存用户与物品图表示 ============
    def forward(self):
        device = self.embedding_item.weight.device
        user_init = self.embedding_user.weight                              # (U, D)

        if self.fusion_mode == 'none':
            # 纯 ID：一次传播
            item_init = self._item_init_id_only()
            user_embedding, item_embedding = self._propagate(user_init, item_init)

        elif self.fusion_mode == 'early':
            # 前融合：先做模态映射，再作为图的初始 item 表示
            item_init = self._item_init_early()
            user_embedding, item_embedding = self._propagate(user_init, item_init)

        else:  # 'late'
            # 后融合：两条路径分别图传播，最后在图传播结果处做 α 加权（仅对 item）
            item_init_id = self._item_init_id_only()
            item_init_mm = self._item_init_early()

            # 两次传播（共享同一 user_init）
            user_id_emb,  item_id_emb  = self._propagate(user_init, item_init_id)
            user_mm_emb,  item_mm_emb  = self._propagate(user_init, item_init_mm)

            alpha = torch.sigmoid(self.alpha_param).to(device)  # 标量
            item_embedding = alpha * item_id_emb + (1.0 - alpha) * item_mm_emb
            # 用户侧：为最小改动与稳定性，采用 ID 路径的 user 表示（也可两路再平均/融合）
            user_embedding = user_id_emb

        # 缓存（评测/导出用）
        self.user_emb = user_embedding.detach()
        self.item_emb = item_embedding.detach()
        return user_embedding, item_embedding

    # ============ Getter（评测/召回） ============
    def get_users_embedding(self, user_ids, l2_norm=False):
        u_vec = self.user_emb[user_ids]
        if l2_norm:
            u_vec = F.normalize(u_vec, p=2, dim=1)
        return u_vec

    def get_items_embedding(self, item_ids, l2_norm=False):
        i_vec = self.item_emb[item_ids]
        if l2_norm:
            i_vec = F.normalize(i_vec, p=2, dim=1)
        return i_vec

    # ============ 导出 ============
    def save_embeddings(self, num_users, num_items, device, save_dir='./embeddings', l2_norm=L2_NORM):
        import os, faiss
        os.makedirs(save_dir, exist_ok=True)
        self.eval().to(device)

        user_ids = torch.arange(num_users, dtype=torch.long, device=device)
        item_ids = torch.arange(num_items, dtype=torch.long, device=device)

        with torch.no_grad():
            user_embeds = self.get_users_embedding(user_ids, l2_norm=l2_norm)
            item_embeds = self.get_items_embedding(item_ids, l2_norm=l2_norm)

        user_embeds = user_embeds.cpu().numpy().astype(np.float32)
        item_embeds = item_embeds.cpu().numpy().astype(np.float32)

        np.save(f"{save_dir}/user_embeddings.npy", user_embeds)
        np.save(f"{save_dir}/item_embeddings.npy", item_embeds)

        dim = item_embeds.shape[1]
        index = faiss.IndexFlatIP(dim)
        index.add(item_embeds)
        faiss.write_index(index, f"{save_dir}/item_index.faiss")
        print("Saved user/item embeddings and FAISS index.")


In [44]:
import torch
import numpy as np

def train_model(model,
                train_df,
                num_items,
                epochs=EPOCHS,
                batch_size=BATCH_SIZE,
                lr=LR,
                device=None):
    """
    训练 LightGCN (或其它 BPR 模型) 的通用函数
    ------------------------------------------------
    • train_df      : pandas DataFrame，含 user_id / item_id
    • num_items     : 物品总数
    • device        : torch.device；默认为 'cuda' (若可用) 否则 'cpu'
    • max_grad_norm : 梯度裁剪阈值；避免梯度爆炸，可选
    """
    # -------- 设备 ----------
    if device is None:
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    model = model.to(device)
    if hasattr(model, "adjacency"):               # adjacency 可能是稀疏张量
        model.adjacency = model.adjacency.to(device)

    # -------- 优化器 ----------
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    train_loader = customdataset.build_train_loader_inbatch(train_df, batch_size=batch_size,user_col=user_id, item_col=item_id)

    # -------- 训练循环 ----------
    for epoch in range(1, epochs + 1):
        model.train()
        dt_start = datetime.now()
        epoch_loss = 0.0

        for batch in train_loader:
            user_ids, pos_item_ids = batch
            user_ids = user_ids.to(device).long()
            pos_item_ids = pos_item_ids.to(device).long()


            # 1. 前向传播（返回 user / item 向量）
            user_emb, item_emb = model()
            u_vec = user_emb[user_ids]
            i_vec = item_emb[pos_item_ids]

            # 2. 得分矩阵：每个 user 对所有正 item 的打分
            logits = torch.matmul(u_vec, i_vec.T)  # shape: (B, B)

            # 3. 构造标签：每个 user 的正确 item 在对角线（即位置 i）
            labels = torch.arange(logits.size(0), device=device)  # [0, 1, ..., B-1]

            # 4. Cross Entropy Loss
            loss = F.cross_entropy(logits, labels)

            # 5. 反向传播
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        # 日志
        avg_loss = epoch_loss / len(train_loader)
        dt_end = datetime.now()
        dt = dt_end - dt_start

        print(f"[Epoch {epoch:02d}/{epochs}] avg InBatch Softmax Loss = {avg_loss:.4f}, time = {dt.total_seconds():.2f}s")
        if(model.fusion_mode=='late'):
            print('alpha:', torch.sigmoid(model.alpha_param).item())
    return


In [45]:
model = LightGCN(num_users,num_items,embedding_dim=EMBEDDING_DIM,n_layers=N_LAYERS,adjacency=adj_torch)
model.to(device)
train_model(model=model,epochs=EPOCHS, train_df=train_df,num_items=num_items,batch_size=BATCH_SIZE)

[Epoch 01/8] avg InBatch Softmax Loss = 5.5854, time = 3.65s
[Epoch 02/8] avg InBatch Softmax Loss = 4.9674, time = 3.65s
[Epoch 03/8] avg InBatch Softmax Loss = 4.6006, time = 3.65s
[Epoch 04/8] avg InBatch Softmax Loss = 4.3015, time = 3.62s
[Epoch 05/8] avg InBatch Softmax Loss = 4.0619, time = 3.66s
[Epoch 06/8] avg InBatch Softmax Loss = 3.8671, time = 3.62s
[Epoch 07/8] avg InBatch Softmax Loss = 3.6975, time = 3.65s
[Epoch 08/8] avg InBatch Softmax Loss = 3.5447, time = 3.63s


In [46]:
model.save_embeddings(num_users=num_users,num_items=num_items,device=device,save_dir=save_dir)

Saved user/item embeddings and FAISS index.


In [47]:
test_loader = customdataset.build_test_loader(test_df, num_items ,user_col = user_id, item_col = item_id, batch_size=1024, num_workers=num_workers)
item_pool = list(range(num_items))
faiss_index = faiss.read_index(f"{save_dir}/item_index.faiss")

In [48]:
hr_r, ndcg_r = evaluate.evaluate_random(test_loader, item_pool ,top_k=top_k)
print(f"Random HR@{top_k} = {hr_r:.4f}, NDCG@{top_k} = {ndcg_r:.4f}")
hr_p, ndcg_p = evaluate.evaluate_popular(test_loader, train_df,top_k=top_k)
print(f"Popular HR@{top_k} = {hr_p:.4f}, NDCG@{top_k} = {ndcg_p:.4f}")
hr_m, ndcg_m = evaluate.evaluate_model(test_loader, model, faiss_index, device,top_k=top_k)
print(f"Model   HR@{top_k} = {hr_m:.4f}, NDCG@{top_k} = {ndcg_m:.4f}")


Random HR@10 = 0.0005, nDCG@10 = 0.0002
Popular HR@10 = 0.0029, nDCG@10 = 0.0014
Model   HR@10 = 0.0294, nDCG@10 = 0.0115
